In [16]:
import os.path
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile

import torch
import numpy as np
from random import shuffle
from torch import from_numpy
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import BatchSampler
from torch.utils.data import RandomSampler
from sklearn.model_selection import train_test_split


import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import wandb


### Movielens Dataset. 

In [2]:
basedir = f'/home/dewan/codespace/news-recommender/data/ml-1m'

users = pd.read_csv(f'{basedir}/users.dat', delimiter="::", names=['user', 'gender', 'age', 'occupation', 'zipcode'])
movies = pd.read_csv(f'{basedir}/movies.dat', delimiter="::", names=['id', 'title', 'genres_str'], encoding='latin-1')
movies['genres'] = movies['genres_str'].apply(lambda gs: {key: 1.0 for key in gs.split('|')})
ratings = pd.read_csv(f'{basedir}/ratings.dat', delimiter="::", names = ['user', 'item', 'rating', 'timestamp'])
ratings['rank'] = ratings.groupby("user")['timestamp'].rank(ascending=True)
ratings['training'] = np.random.random(len(ratings)) < 0.90
print(users.columns)
print(movies.columns)
print(ratings.columns)

/tmp/ipykernel_14017/2176144171.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv(f'{basedir}/users.dat', delimiter="::", names=['user', 'gender', 'age', 'occupation', 'zipcode'])
/tmp/ipykernel_14017/2176144171.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv(f'{basedir}/movies.dat', delimiter="::", names=['id', 'title', 'genres_str'], encoding='latin-1')
/tmp/ipykernel_14017/2176144171.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpret

Index(['user', 'gender', 'age', 'occupation', 'zipcode'], dtype='object')
Index(['id', 'title', 'genres_str', 'genres'], dtype='object')
Index(['user', 'item', 'rating', 'timestamp', 'rank', 'training'], dtype='object')


In [3]:
df = ratings.merge(users, on='user')
df = df.merge(movies, left_on='item', right_on='id')
df

,user,item,rating,timestamp,rank,training,gender,age,occupation,zipcode,id,title,genres_str,genres
0,1,1193,5,978300760,11.0,True,F,1,10,48067,1193,One Flew Over the Cuckoo's Nest (1975),Drama,{'Drama': 1.0}
1,2,1193,5,978298413,8.5,True,M,56,16,70072,1193,One Flew Over the Cuckoo's Nest (1975),Drama,{'Drama': 1.0}
2,12,1193,4,978220179,15.5,True,M,25,12,32793,1193,One Flew Over the Cuckoo's Nest (1975),Drama,{'Drama': 1.0}
3,15,1193,4,978199279,152.5,True,M,25,7,22903,1193,One Flew Over the Cuckoo's Nest (1975),Drama,{'Drama': 1.0}
4,17,1193,5,978158471,2.5,True,M,50,1,95350,1193,One Flew Over the Cuckoo's Nest (1975),Drama,{'Drama': 1.0}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,51.0,True,M,18,17,47901,2198,Modulations (1998),Documentary,{'Documentary': 1.0}
1000205,5675,2703,3,976029116,549.0,True,M,35,14,30030,2703,Broken Vessels (1998),Drama,{'Drama': 1.0}
1000206,5780,2845,1,958153068,3.5,True,M,18,17,92886,2845,White Boys (1999),Drama,{'Drama': 1.0}
1000207,5851,3607,5,957756608,13.0,True,F,18,20,55410,3607,One Little Indian (1973),Comedy|Drama|Western,"{'Comedy': 1.0, 'Drama': 1.0, 'Western': 1.0}"


In [4]:
# Compute cardinalities
n_features = df.user.max() + 1 + df.item.max() + 1
n_user = df.user.max() + 1
n_item = df.item.max() + 1
n_rank = df['rank'].max() + 1
n_occu = df['occupation'].max() + 1
print('n_item', n_item)
print('n_user', n_user)
print('n_featuers', n_features)
print('n_occu', n_occu)
print('n_rows', len(df))

n_item 3953
n_user 6041
n_featuers 9994
n_occu 21
n_rows 1000209


In [5]:
def split(subset):
    feat_cols = ['user', 'item', 'rank', 'occupation']
    out_cols = ['rating']
    features = subset[feat_cols]
    features_dict = list(subset['genres'].values)
    outcomes = subset[out_cols]
    features = features.values.astype(np.int32)
    outcomes = outcomes.values.astype(np.float32)
    both = subset[feat_cols + out_cols]
    return features, outcomes, both, features_dict


train_x, train_y, train_xy, train_dict = split(df[df.training])
test_x, test_y, test_xy, test_dict = split(df[~df.training])

np.savez("data/movielens_dataset.npz", train_x=train_x, train_y=train_y,
         train_xy=train_xy, test_x=test_x, test_y=test_y, test_xy=test_xy,
         train_dict=train_dict, test_dict=test_dict,
         n_user=n_user, n_item=n_item, n_ranks=n_rank, n_occu=n_occu)

In [6]:
!wandb login 2be51600748a2feef71c083ae7f0bd21375c3eb6

wandb: Appending key for api.wandb.ai to your netrc file: /home/dewan/.netrc


In [9]:
## Transform data


fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
X_test = fh['test_x'].astype(np.int64)
Y_test = fh['test_y']

X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y)


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(f'X_train = {X_train}')
print(' ')
print(f'Y_train = {Y_train}')



X_train = [[5136  357   26   12]
 [3569 1327   52    2]
 [5708 1199   43    1]
 ...
 [3989 2407  144   12]
 [1667 2737  498   16]
 [1088 2366   21   10]]
 
Y_train = [[4.]
 [3.]
 [3.]
 ...
 [4.]
 [2.]
 [5.]]


In [11]:
def dataloader(*arrs, batch_size=32):
    dataset = TensorDataset(*arrs)
    bs = BatchSampler(RandomSampler(dataset), 
                      batch_size=batch_size, drop_last=False)
    return DataLoader(dataset, batch_sampler=bs, num_workers=8)
 
train = dataloader(from_numpy(X_train), from_numpy(Y_train))
test = dataloader(from_numpy(X_test), from_numpy(Y_test))
val = dataloader(from_numpy(X_val), from_numpy(Y_val))



In [12]:

class AbstractModel(pl.LightningModule):
    def step(self, batch, batch_nb, prefix='train', add_reg=True):
        input, target = batch
        prediction = self.forward(input)
        loss, log = self.loss(prediction, target)
        
        if add_reg:
            loss_reg, log_ = self.reg()
            loss = loss + loss_reg
            log.update(log_)
        log[f'{prefix}_loss'] = loss
        return {f'{prefix}_loss': loss, 'loss':loss, 'log': log}

    def training_step(self, batch, batch_nb):
        return self.step(batch, batch_nb, 'train')
    
    def test_step(self, batch, batch_nb):
        # Note that we do *not* include the regularization loss
        # at test time
        return self.step(batch, batch_nb, 'test', add_reg=False)    
    
    def validation_step(self, batch, batch_nb):
        return self.step(batch, batch_nb, 'val', add_reg=False)    
    
    def test_epoch_end(self, outputs):
        loss_mean = torch.stack([x['test_loss'] for x in outputs]).mean()
        log = {'test_loss': loss_mean}
        return {'avg_test_loss': loss_mean, 'log': log}

    def validation_epoch_end(self, outputs):
        loss_mean = torch.stack([x['val_loss'] for x in outputs]).mean()
        log = {'val_loss': loss_mean}
        return {'avg_val_loss': loss_mean, 'log': log}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [17]:
def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, batch_size=128):
        super().__init__()
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # equivalent:
        # self.user.weight[user_id, :]
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        # shape vector_user is (batch_size, k)
        # vector_user * vector_item is shape (batch_size, k)
        # sum(vector_user * vector_item is shape, dim=1) (batch_size)
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        return ui_interaction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        # target is (batchsize, 1)
        # target.squeeze (batchsize, )
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse, {"mse": loss_mse}
    
    def reg(self):
        # Compute L2 reularization over user (P) and item (Q) matrices
        reg_user =  l2_regularize(self.user.weight) * self.c_vector
        reg_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        log = {"reg_user": reg_user, "reg_item": reg_item}
        total = reg_user + reg_item
        return total, log

In [25]:
# from pytorch_lightning.loggers.wandb import WandbLogger, finish

batch_size = 1024
k = 5
c_vector = 1e-5
model = MF(n_user, n_item, k=k, c_vector=c_vector,
          batch_size=batch_size)

# wandb.finish()
# add a logger
logger = wandb.WandbLogger(name="matrix_factor", project="movielens_recsys")

trainer = pl.Trainer(max_epochs=20, logger=logger) 

/home/dewan/codespace/news-recommender/nrs/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:352: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [26]:
trainer.fit(model, train, val)


/home/dewan/codespace/news-recommender/nrs/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory ./movielens_recsys/36ay6mi3/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name | Type      | Params
-----------------------------------
0 | user | Embedding | 30.2 K
1 | item | Embedding | 19.8 K
-----------------------------------
50.0 K    Trainable params
0         Non-trainable params
50.0 K    Total params
0.200     Total estimated model params size (MB)


Validation DataLoader 0:  11%|█         | 749/7033 [00:02<00:24, 257.65it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 0:  81%|████████  | 22852/28130 [01:32<00:21, 246.55it/s, loss=14.2, v_num=6mi3]


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 19:  95%|█████████▍| 26685/28130 [01:26<00:04, 306.99it/s, loss=1.07, v_num=6mi3]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 19:  99%|█████████▉| 27922/28130 [01:30<00:00, 309.91it/s, loss=1.07, v_num=6mi3]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
results = trainer.test(model)
mse = results['avg_test_loss']
rmse = np.sqrt(mse)
rmse
